In [2]:
import pandas as pd
# Imports polling data and filters for test years
polling_df = pd.read_csv('../processed_data/2004_to_2019_polling_cleaned.csv')
# Filters to 2019 election
election_2019_data = polling_df[polling_df['next_elec_date'] == '2019-12-12']

In [4]:
# Filters to 2019 election
election_2019_data = polling_df[polling_df['next_elec_date'] == '2019-12-12']

In [5]:
# Filters to forecast columns
forecast_columns = ['BRX_FC', 'CON_FC', 'GRE_FC', 'LAB_FC', 'LIB_FC', 'NAT_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC']
forecast_data = election_2019_data[forecast_columns]

In [19]:
forecast_data

,BRX_FC,CON_FC,GRE_FC,LAB_FC,LIB_FC,NAT_FC,OTH_FC,PLC_FC,SNP_FC,UKI_FC
2820,NaN,39.0,NaN,45.0,7.0,NaN,2.0,1.0,3.0,3.0
2821,NaN,41.0,1.0,44.0,6.0,NaN,3.0,1.0,3.0,2.0
2822,NaN,41.0,1.0,46.0,6.0,NaN,0.0,1.0,3.0,2.0
2823,NaN,39.0,2.0,45.0,5.0,NaN,0.0,0.0,3.0,5.0
2824,NaN,41.0,2.0,40.0,7.0,NaN,6.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3255,2.0,44.0,3.0,33.0,12.0,NaN,6.0,NaN,NaN,NaN
3256,3.0,44.0,3.0,32.0,13.0,NaN,1.0,0.0,4.0,NaN
3257,2.0,45.0,2.0,33.0,12.0,NaN,1.0,0.0,4.0,NaN
3258,4.0,43.0,3.0,34.0,11.0,NaN,NaN,0.0,4.0,NaN


In [8]:
mean_forecast = forecast_data.mean()

In [21]:

mean_forecast

BRX_FC    10.652406
CON_FC    37.268182
GRE_FC     3.725581
LAB_FC    33.959091
LIB_FC    11.613636
NAT_FC     2.538462
OTH_FC     1.311213
PLC_FC     0.601190
SNP_FC     3.775463
UKI_FC     3.344920
dtype: float64

In [29]:
total = mean_forecast.sum()

In [32]:
national_forecast = {k[:3]:(v/total)*100 for k,v in zip(mean_forecast.index, mean_forecast.values)}

In [33]:
national_forecast

{'BRX': 9.791701691233214,
 'CON': 34.256946707559216,
 'GRE': 3.4245578155056258,
 'LAB': 31.215227326768495,
 'LIB': 10.67526513450589,
 'NAT': 2.3333561606657462,
 'OTH': 1.205268014756199,
 'PLC': 0.5526148338661842,
 'SNP': 3.470408997937417,
 'UKI': 3.074653317201996}

In [34]:
# Imports test bias data and filters for test years, (this test data set removes 2019 bias)
test_bias = pd.read_csv('../processed_data/removes_2019_constituency_bias_dataframe.csv')

In [39]:
test_bias['BRX_Bias_Score'] = 0

In [40]:
def calculate_predicted_vote_share(national_forecast, bias_scores):
    predicted_vote_share = {}
    for party, forecast_share in national_forecast.items():
        bias_score = bias_scores[party + '_Bias_Score']
        predicted_vote_share[party] = forecast_share + bias_score
    return predicted_vote_share

# Function to determine the winning party in a constituency
def determine_winner(predicted_vote_share):
    return max(predicted_vote_share, key=predicted_vote_share.get)

# Function to determine the winner for each constituency using the constituency bias DataFrame
def determine_winners(national_forecast, test_bias):
    winners = []
    for index, row in test_bias.iterrows():
        predicted_vote_share = calculate_predicted_vote_share(national_forecast, row)
        winner = determine_winner(predicted_vote_share)
        winners.append({'Constituency': row['Constituency'], 'Winner': winner})
    return pd.DataFrame(winners)

In [41]:
winners_df = determine_winners(national_forecast, test_bias)


In [46]:
winners_df

,Constituency,Winner
0,Aberavon,LAB
1,Aberconwy,CON
2,Aberdeen North,SNP
3,Aberdeen South,LAB
4,Aberdeenshire West and Kincardine,CON
...,...,...
726,Ynys Mon,LAB
727,York Central,LAB
728,York Outer,CON
729,"York, City of",LAB


In [53]:
sum = winners_df['Winner'].value_counts()

In [54]:
sum

Winner
CON    339
LAB    328
LIB     23
OTH     19
SNP     16
PLC      5
GRE      1
Name: count, dtype: int64